# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?
Naive Bayes assume that the effect of the value of a predictor (x) on a given class (c) is independent of the values of other predictors. This assumption is called class conditional independence.

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?
One way to obtain a point estimate is to choose the value of x that maximizes the posterior PDF (or PMF). This is called the maximum a posteriori (MAP) estimation. Maximum a posteriori (MAP) estimate of X given Y=y is the value of x that maximizes the posterior PDF。
If the prior probability is the same for all the hypothesis then MAP = ML

#### 3. What is support vector in SVM?
Support vectors are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane. 

#### 4. What is the intuition behind SVM ?
Find the boundary for classification that has the maximum margin

#### 5. Shortly describ what 'random' means in random forest ?
Random means rando forest use a random subset of samples and a random sample of features in each tree.

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:
###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  
###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

In [35]:
import pandas as pd
path='/Users/zixiawang/school/kaikeba/作业笔记/开课吧NLP/NLP_assignment/Project1/Unsupervise_news_abstract/raw_data'
df=pd.read_csv(path+'/sqlResult_1558435.csv',encoding='GB18030')
print (len(df))

89611


In [12]:
source=df.source.value_counts(2).to_frame().reset_index()
source

,index,source
0,新华社,0.877825
1,微博,0.027642
2,中国证券报?中证网,0.006004
3,中国新闻网,0.005859
4,参考消息网,0.004296
...,...,...
969,谈资,0.000011
970,CNR国防时空,0.000011
971,凤凰号,0.000011
972,新浪财经@,0.000011


In [33]:
df['y']=df.apply(lambda x:  1 if len(re.findall('新华社',str(x.source)))>0  else 0, axis=1)
#lambda x : True if (x > 10 and x < 20) else False
df['y'].value_counts(2)

1    0.87795
0    0.12205
Name: y, dtype: float64

In [34]:
df[df.y==1].head()

,id,author,source,content,feature,title,url,y
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1
39,89578,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....,1
65,89552,NaN,新华社@,从百度、支付宝到App?“高考志愿填报”五花八门\r\n顾女士近来减少了刷朋友圈的频率，一有...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""5"",...",高考志愿填报：眼花缭乱的App、上万元咨询靠谱吗？,http://www.cnbeta.com/articles/tech/623637.htm,1
95,89522,NaN,新华社,新华社韩国济州6月18日电综述：亚投行第二届年会三大亮点\r\n新华社记者 耿学鹏 严蕾\r...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",综述：亚投行第二届年会三大亮点,http://china.huanqiu.com/hot/2017-06/10866439....,1
113,89504,NaN,新华社,新华社北京6月18日电 经军委领导批准，《军营理论热点怎么看·2017》日前印发全军。\r\...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",《军营理论热点怎么看·2017》印发全军,http://china.huanqiu.com/hot/2017-06/10866447....,1


In [4]:
df.content[0].replace('\r\n', '')

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。当然，关于MIUI 9的确切信息，我们还是等待官方消息。'

In [37]:
df['title_content']=df['title']+df['content']
df['title_content'].to_csv('../segment/news_text.txt', index=False)

/Users/zixiawang/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [44]:
import re
from collections import Counter
## clean up the segement text before count the frequency

def get_chinese_number_letters(train_file):
    train = train_file
    train = open(train,encoding="utf8", errors='ignore').read().lower()
    #[\u00C0-\u017F] Unicode range for all Latin characters
    #[\u4e00-\u9fff] Unicode range for all chinese characters
    chinese_only=re.findall(r'([\u4e00-\u9fff]+|[A-Z]+|[0-9]+|[a-z]+| [ ])',train)
    
    #chinese_only=re.findall('[\w\s]',train)
    print ('First 20 characters of the train file is :', ' '.join(chinese_only)[:20])
    return ' '.join(chinese_only)

def get_1_gram_count(text):
    print ('Length of cleaned train file: ', len(text))
    TOKENS = text.split(' ')
    print ('Train file contains ', len(TOKENS), ' of words(tokens).')
    _1_gram_word_counts = Counter(TOKENS)
    return _1_gram_word_counts

In [45]:
import time
start_time = time.time()
train_file='../raw/news_text.txt'
clean_file=get_chinese_number_letters(train_file)
_1_gram_word_counts=get_1_gram_count(clean_file)
print ("My program took", time.time() - start_time, "to run")

First 20 characters of the train file is : 小米 miui 9 首批机型曝光 共计 
Length of cleaned train file:  40461622
Train file contains  6032982  of words(tokens).
My program took 4.561856746673584 to run


In [51]:
text_file = open("../segment/news_segment.txt", "w")
text_file.write(clean_file)
text_file.close()

In [52]:
## use model = word2vec.Word2Vec.load("../models/wiki_news.model") to load the model if you need to re-train the model
# 将Word转换成Vec，然后计算相似度 
from gensim.models import word2vec
import multiprocessing

# 如果目录中有多个文件，可以使用PathLineSentences
sentences = word2vec.PathLineSentences('../segment/')
# 设置模型参数，进行训练
import time
start_time = time.time()
model = word2vec.Word2Vec(sentences, size=300, window=5, min_count=2,workers=multiprocessing.cpu_count())
print ("My program took", time.time() - start_time, "to run")

My program took 36.84010171890259 to run


In [53]:
len(model.wv.vocab.items())


477373

In [57]:
list(model.wv.vocab.keys())[:10]

['小米', 'miui', '9', '共计', '15', '6', '月', '12', '日', '起']

In [56]:
print(model.wv.similarity('中国', '美国'))
#save the model

0.67421377


In [58]:
model.save('../models/news.model')
from gensim.models import word2vec
model = word2vec.Word2Vec.load("../models/news.model")

### Output word vectors to wiki_news_word_vector.txt for SIF embeddings

In [ ]:
word_list=list(model.wv.vocab.keys())
word_vector_file= open("wiki_news_word_vector.txt","a")
for a, i in enumerate(word_list):
    #print (i + ' '+' '.join(map(str, model.wv[i[0]]))  )
    if a%100000==0:
        print (a)
    string=(i + ' '+' '.join(map(str, model.wv[i]))+'\n'  )
    word_vector_file.write(string)
word_vector_file.close()

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).